# LlamaIndex-RAG Community Q&A system

## [Github](https://github.com/run-llama/llama_index)

In [1]:
!pip install llama_index transformers accelerate bitsandbytes -q

In [2]:
!pip install llama-index-llms-huggingface -q
!pip install llama-index-embeddings-langchain -q

## Setup LLM

In [3]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

/home/rkuo/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
# set quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

### Load LLM model

In [5]:
model_name = "taide/Llama3-TAIDE-LX-8B-Chat-Alpha1"

llm = HuggingFaceLLM(
    model_name=model_name,
    tokenizer_name=model_name,
    query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n"),   
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},    
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95 , "do_sample": True},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Choosing the embedding model

### BAAI/bge-small-en-v1.5
### shibingtext2vec-bge-large-chinese
### GanymedeNil/text2vec-large-chinese

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

#lc_embed_model = HuggingFaceEmbeddings(model_name=BAAI/bge-small-en-v1.5") # For English
lc_embed_model = HuggingFaceEmbeddings(model_name="GanymedeNil/text2vec-large-chinese") # For Chinese
embed_model = LangchainEmbedding(lc_embed_model)

No sentence-transformers model found with name GanymedeNil/text2vec-large-chinese. Creating a new one with MEAN pooling.


In [7]:
from llama_index.core import ServiceContext, Document
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

/tmp/ipykernel_43316/3305293978.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)


## Building a local VectorIndex

In [8]:
#from llama_index.readers import BeautifulSoupWebReader

#url = "https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots"
#documents = BeautifulSoupWebReader().load_data([url])

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./docs/community/").load_data()
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

## Generate Response

In [10]:
from llama_index.legacy.response.notebook_utils import display_response

In [11]:
query_engine = vector_index.as_query_engine()

In [12]:
response = query_engine.query("區權會應該多久開一次?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2024-05-13 05:57:32.758408: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 05:57:32.776668: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 05:57:33.145850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


**`Final Response:`** 根據公寓大廈管理條例第二十五條規定，區分所有權人會議應每半年至少召開一次。因此，區權會應該每半年開一次。若有特殊情況或需要，召集人可視情況召開臨時會議，但需遵循相關規定。若會議目的涉及專有部分的約定共用事項，應先取得該專有部分的區分所有權人書面同意，方可成為議案。會議中涉及到專有部分的變更時，應經使用該專有部分的區分所有權人同意，除非該約定專用已違反公共利益，經管理委員會或管理負責人訴請法院判決確定者，不在此限。
此外，會議中涉及到專有部分的承租者或使用者時，該等承租者或使用者可經該專有部分的區分所有權人同意後，列席區分所有權人會議陳述意見。最後，區分所有權人因故無法出席會議時，可

In [13]:
response = query_engine.query("區權會投票是否要按所有權比例計票?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**`Final Response:`** 根據公寓大廈管理條例第27條規定，各專有部分的區分所有權人有一表決權，數人共有一專有部分者，該表決權應推由一人行使。在計算出席人數與表決權時，若任一區分所有權人之區分所有權占全部區分所有權五分之一以上者，或任一區分所有權人所有之專有部分之個數超過全部專有部分個數總合之五分之一以上者，其超過部分不予計算。因此，在區分所有權人會議中，表決權是按所有權比例計票，但超過一定比例的所有權部分，其超過部分將不予計算。 [/